# Voila dashboard demo

Running:

```bash
voila app-voila.ipynb
```

Resources:

* [introduction to ipywidgets interact](https://ipywidgets.readthedocs.io/en/latest/examples/Using%20Interact.html)
* [Widgets that depend on each other](https://ipywidgets.readthedocs.io/en/latest/examples/Using%20Interact.html#Arguments-that-are-dependent-on-each-other)
* [voila github repo](https://github.com/voila-dashboards/voila)


In [1]:
import ipywidgets as widgets
from ipywidgets import interact
from plotnine import *

from siuba import _, filter, summarize, mutate
from siuba.data import mtcars
import pandas as pd
from plotly import tools

import warnings
warnings.filterwarnings("ignore")

jobs_gender = pd.read_csv("https://raw.githubusercontent.com/rfordatascience/tidytuesday/master/data/2019/2019-03-05/jobs_gender.csv")


In [2]:
def plot_pay_gap_2016(jobs_gender, major_category):
    return (
        jobs_gender
        >> filter(_.year == 2016, _.total_workers >= 20000)
        >> filter(_.major_category == major_category)
        #     >> arrange(desc(wage_percent_of_male))
        >> mutate(
            percent_female=_.workers_female / _.total_workers,
            wage_percent_female=_.total_earnings_female / _.total_earnings_male,
        )
        >> ggplot(
            aes(
                "percent_female",
                "wage_percent_female",
                color="minor_category",
                size="total_workers",
                label="occupation",
            )
        )
        + geom_point()
        + scale_size_continuous(range=[1, 10], guide=False)
        + labs(
            x="% of workforce reported as female",
            y="% of median female salary / median male",
            title="Gender disparity and pay gap in 2016",
            subtitle="Only occupations with at least 20,000 workers total",
            color="Minor category",
        )
        #        scale_x_continuous(labels = percent_format()) +
        #        scale_y_continuous(labels = percent_format())
    )

# p = plot_pay_gap_2016(jobs_gender, "Management, Business, and Financial")
# tools.mpl_to_plotly(p.draw())

In [5]:
w = widgets.Dropdown(options=jobs_gender.major_category.unique(), value=None)

w2 = widgets.Dropdown(options=[], value=None)

def update_minor_options(*args):
    print('updating')
    w2.options = jobs_gender \
        .loc[lambda d: d.major_category == w.value, 'minor_category'] \
        .unique()

w.observe(update_minor_options, 'value')

@interact(major_category=w, minor_category = w2)
def f(major_category, minor_category):
    filtered_jobs = filter(
        jobs_gender,
        _.major_category == major_category,
        _.minor_category == minor_category
    )
    
    p = plot_pay_gap_2016(filtered_jobs, major_category)
    return tools.mpl_to_plotly(p.draw())


interactive(children=(Dropdown(description='major_category', options=('Management, Business, and Financial', '…